In [1]:
from lxml import etree
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk import WordNetLemmatizer
from collections import defaultdict
import string
import re
import nltk
import pandas as pd

In [2]:
tree = etree.parse('data/news.xml')
root = tree.getroot()
corpus = root[0]

wnl = WordNetLemmatizer()
data = pd.DataFrame({'Title':[], 'Text':[]})

My custom model using tf-idf and parsing the data manually

In [3]:
for news in corpus:
    title = news[0].text
    text = news[1].text
    
    text_tokenized = word_tokenize(text.lower())
    text_lemmatized = [wnl.lemmatize(word) for word in text_tokenized]
    text_lemmatized = [wnl.lemmatize(word) for word in text_lemmatized if word not in stopwords.words('english')]
    punct = f'^[{re.escape(string.punctuation)}]+$'
    no_punctuation = [re.sub(punct, '', word) for word in text_lemmatized]
    nouns = [word for word in no_punctuation if nltk.pos_tag([word])[0][1] == 'NN'] 
    data = data.append({'Title': title, 'Text': ' '.join(nouns)}, ignore_index=True)
    
    words_counter = defaultdict(int)
    for word in nouns:
        words_counter[word] += 1
        
    top5 = sorted(words_counter.items(), key=lambda x: (x[1], x[0]), reverse=True)[1:6]
    print(title + ':')
    for word, count in top5:
        print(word, end=' ')
        
    print('\n')

Brain Disconnects During Sleep:
sleep cortex consciousness tononi activity 

New Portuguese skull may be an early relative of Neandertals:
skull fossil europe year trait 

Living by the coast could improve mental health:
health mental coast research living 

Did you knowingly commit a crime? Brain scans could tell:
brain study wa suitcase result 

Computer learns to detect skin cancer more accurately than doctors:
dermatologist skin melanoma year cnn 

US economic growth stronger than expected despite weak demand:
u quarter ha year rate 

Microsoft becomes third listed US firm to be valued at $1tn:
microsoft share cloud market ha 

Apple's Siri is a better rapper than you:
siri wa time rhyme ha 

Netflix viewers like comedy for breakfast and drama at lunch:
netflix show day comedy viewer 

Loneliness May Make Quitting Smoking Even Tougher:
smoking loneliness study smoke quit 



In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(input='content')

tfidf_matrix = vectorizer.fit_transform(data['Text'])

In [5]:
tf_Idf = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names())

/home/wojtek/PycharmProjects/Key Terms Extraction/.idea/VirtualEnvironment/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


The results of sklearn's model

In [6]:
for i in range(10):
    print(data.Title[i])
    print(' '.join(tf_Idf.loc[i, :].nlargest(5).index))
    print()

Brain Disconnects During Sleep
cortex sleep consciousness tononi communication

New Portuguese skull may be an early relative of Neandertals
skull europe fossil genus trait

Living by the coast could improve mental health
health coast mental living household

Did you knowingly commit a crime? Brain scans could tell
brain suitcase study behavior contraband

Computer learns to detect skin cancer more accurately than doctors
dermatologist melanoma skin cnn benign

US economic growth stronger than expected despite weak demand
growth rate quarter economy analyst

Microsoft becomes third listed US firm to be valued at $1tn
microsoft share cloud market company

Apple's Siri is a better rapper than you
siri rhyme lyric mc producer

Netflix viewers like comedy for breakfast and drama at lunch
netflix comedy day breakfast documentary

Loneliness May Make Quitting Smoking Even Tougher
smoking loneliness smoke lead quit

